In [ ]:
# clone the repository
!git clone https://github.com/jkuhlemann/dc_tts.git

Cloning into 'dc_tts'...
remote: Enumerating objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168
Receiving objects: 100% (168/168), 3.13 MiB | 3.95 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
#create folder necessary for everything to work
!mkdir /content/dc_tts/samples
!mkdir /content/dc_tts/logdir
!mkdir /data/
!mkdir /data/private/
!mkdir /data/private/voice/

In [ ]:
# download pre-trained model
import urllib.request

url = "https://www.dropbox.com/s/1oyipstjxh2n5wo/LJ_logdir.tar?dl=1"  

u = urllib.request.urlopen(url)
data = u.read()
u.close()
 
with open('/content/dc_tts/LJ_logdir.tar', "wb") as f :
    f.write(data)

In [ ]:
#extract the pre-trained model
import zipfile
with zipfile.ZipFile('/content/dc_tts/LJ_logdir.tar', 'r') as zip_ref:
    zip_ref.extractall('/content/dc_tts/')

In [ ]:
#download dataset 
url = 'https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2'
urllib.request.urlretrieve(url, '/content/dc_tts/LJSpeech-1.1.tar.bz2')

('/content/dc_tts/LJSpeech-1.1.tar.bz2',
 <http.client.HTTPMessage at 0x7f7ca7141ad0>)

In [49]:
# function to use own sentences for TTS
def sentences_to_synthesize():
  x = input("Do you want to use the example sentences or synthesize your own sentences? Please choose(example/own): ")
  if x == "example":
    print("Good. Just go to the next cell then.")
    pass
  elif x == "own":
    n = 1
    own_sentence = input("Alright. Please write your sentence here: ")
    own_sentence = f"{n}. {own_sentence} \n"
    
    with open('/content/dc_tts/harvard_sentences.txt', 'w') as f:
      f.write(own_sentence)
  while True:
    more = input("Would you like to add another sentence? (y/n): ")
    if more == 'y':
      n = n + 1 
      own_sentence = input("Write your sentence here: ")
      own_sentence = f"{n}. {own_sentence} \n"
      with open('/content/dc_tts/harvard_sentences.txt', 'a') as f:
        f.write(own_sentence)
    elif more == 'n':
      return

  return
  
sentences_to_synthesize()

Do you want to use the example sentences or synthesize your own sentences? Please choose(example/own): own
Alright. Please write your sentence here: test1
Would you like to add another sentence? (y/n): y
Write your sentence here: test2
Would you like to add another sentence? (y/n): y
Write your sentence here: test3
Would you like to add another sentence? (y/n): n


In [ ]:
#extract dataset (takes some time)
!tar -xvf  '/content/dc_tts/LJSpeech-1.1.tar.bz2' -C '/data/private/voice/'

In [ ]:
#manually switch to Tensorflow version 1.x since this is what the repo uses
%tensorflow_version 1.x

In [ ]:
#rename some files so that the code runs
!mv /data/private/voice/LJSpeech-1.1/metadata.csv /data/private/voice/LJSpeech-1.1/transcript.csv
!mv /data/private/voice/LJSpeech-1.1 /data/private/voice/LJSpeech-1.0

In [ ]:
!python /content/dc_tts/prepo.py

In [ ]:
#synthesize the sentences specified in 'harvard_sentences.txt.'
!python /content/dc_tts/synthesize.py